<div class="alert alert-block alert-success">✔️
    

__Комментарий от тимлида №5__

Получилась хорошая работа, развернутый итоговый вывод придает ей вид законченного проекта. Поздравляю с окончанием курса! У тебя получилось справиться со всеми задачами. 
</div>

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

### Задания

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

### Цель исследования

Проанализировать базу данных по заданным параметрам, чтобы сформулировать ценностное предложение для нового продукта. 

Заказчик:  Крупный сервис для чтения книг по подписке.


### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

In [3]:
#создадим функцию для вывода результата, чтобы каждый раз не писать длинную команду
def show_result(query):
    return pd.io.sql.read_sql(query, con = engine)

In [4]:
#выведем первые 5 строк всех таблиц
for table in ['books', 'authors', 'publishers', 'ratings', 'reviews']:
    query = 'SELECT * FROM ' + table + ' LIMIT 5;'
    display(pd.io.sql.read_sql(query, con = engine))

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [5]:
# посчитаем общее количество книг
query = ''' SELECT DISTINCT count(book_id) 
            FROM books
            
'''    
show_result(query)  

,count
0,1000


In [6]:
#посчитайте, сколько книг вышло после 1 января 2000 года
query = ''' SELECT count(book_id)
            FROM books
            WHERE publication_date :: date > '2000-01-01'   
'''    
show_result(query)        

,count
0,819


Из 1000 книг 821 книга вышла после 1 января 2000 года

In [14]:
query = ''' SELECT count(reviews)
            from reviews
'''
show_result(query)

,count
0,2793


In [37]:
#Для каждой книги посчитайте количество обзоров и среднюю оценку
query = ''' SELECT b.title,
            b.book_id,            
            COUNT(DISTINCT review_id),
            AVG(rating)
            FROM reviews as rw
            FULL JOIN ratings AS r on r.book_id=rw.book_id
            RIGHT JOIN books as b on rw.book_id=b.book_id
            GROUP BY b.title, b.book_id          
'''
show_result(query)

,title,book_id,count,avg
0,'Salem's Lot,1,2,3.666667
1,1 000 Places to See Before You Die,2,1,2.500000
2,13 Little Blue Envelopes (Little Blue Envelope...,3,3,4.666667
3,1491: New Revelations of the Americas Before C...,4,2,4.500000
4,1776,5,4,4.000000
...,...,...,...,...
995,Wyrd Sisters (Discworld #6; Witches #2),996,3,3.666667
996,Xenocide (Ender's Saga #3),997,3,3.400000
997,Year of Wonders,998,4,3.200000
998,You Suck (A Love Story #2),999,2,4.500000


In [9]:
# всего ревью
query = ''' SELECT COUNT(review_id)
            FROM reviews        
        
'''
show_result(query) 

,count
0,2793


Для каждой книги посчитано количество обзоров и средняя оценка, для сравнения рассчитаны минимальное количество обзоров = 0, максимальное = 7 у книги Twilight (Twilight #1), минимальная средняя оценка = 1.50 книга Harvesting the Heart, максимальная = 5 у Emily of New Moon (Emily #1)

In [10]:
#Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры
query = ''' SELECT publisher
            FROM publishers
            WHERE publisher_id IN (
                  SELECT publisher_id
                  FROM books
                  WHERE num_pages > 50
                  GROUP BY publisher_id
                  ORDER BY COUNT(book_id) DESC
                  LIMIT 1)
        
'''
show_result(query) 

,publisher
0,Penguin Books


Самое большое количество - 42 книги выпустило издательство 	Penguin Books

In [11]:
#Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками
query = '''SELECT a.author AS aut,
                  AVG(r.rating) AS top_avg
           FROM  authors AS a
           INNER JOIN books AS b on b.author_id = a.author_id
           INNER JOIN ratings AS r ON r.book_id = b.book_id
           WHERE b.book_id in (
                               SELECT book_id
                               FROM ratings
                               GROUP BY book_id
                               HAVING COUNT(rating) >=50
                              )
           GROUP BY author
           ORDER BY top_avg DESC
           LIMIT 1
           
'''
show_result(query)

,aut,top_avg
0,J.K. Rowling/Mary GrandPré,4.287097


В ходе составления запроса получено 19 книг 14 -ти авторов, имеющие 50 и более оценок, из них самая высокая оценка 4.287097 у автора J.K. Rowling/Mary GrandPré. 

In [12]:
#Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок
query = '''SELECT ROUND(AVG(count))
           FROM (
                  SELECT username,
                  COUNT(review_id)
                  FROM reviews
                  WHERE username IN (
                        SELECT username
                        FROM ratings
                        GROUP BY username
                        HAVING COUNT(rating_id) >50
                          )
                  GROUP BY  username
                ) С
'''
show_result(query)            

,round
0,24.0


В ходе составления запроса получено 6 пользователей, поставивших более 50 оценок, в среднем они написали 24 обзора.

- Из 1000 книг 821 книга вышла после 1 января 2000 года
- Для каждой книги посчитано количество обзоров и средняя оценка, для сравнения рассчитаны минимальное количество обзоров = 0, максимальное = 7 у книги Twilight (Twilight #1), минимальная средняя оценка = 1.50 книга Harvesting the Heart, максимальная = 5 у Emily of New Moon (Emily #1)
- Самое большое количество - 42 книги выпустило издательство 	Penguin Books
- В ходе составления запроса получено 19 книг 14 -ти авторов, имеющие 50 и более оценок, из них самая высокая оценка 4.287097 у автора J.K. Rowling/Mary GrandPré. 
- В ходе составления запроса получено 6 пользователей, поставивших более 50 оценок, в среднем они написали 24 обзора.

